In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
train_data = pd.read_csv('C:../../Dataset/home-data-for-ml-course/train.csv')
test_data = pd.read_csv('C:../../Dataset/home-data-for-ml-course/test.csv')

In [3]:
train_data_clean = train_data.dropna(axis=0, subset=['SalePrice'])
y = train_data_clean.SalePrice
X = train_data_clean.drop(['SalePrice'], axis=1)

X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

### Step1. Preprocessing

In [4]:
# Select categorical variables from train data
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and X_train_full[cname].dtype == 'object']

# Select numerical variables from train data
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Select categorical variables and numerical variables from train data
total_selected_cols = categorical_cols + numerical_cols

In [5]:
X_train = X_train_full[total_selected_cols].copy()
X_valid = X_valid_full[total_selected_cols].copy()
X_test = test_data[total_selected_cols].copy()

In [6]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [7]:
# imputation of numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# imputation and one-hot encoding of categorical data
categorical_transformer = Pipeline(steps = [('impute', SimpleImputer(strategy='most_frequent')), ('onehot', OneHotEncoder(handle_unknown='ignore'))])


### Step2. Bundle preprocessing for numerical and categorical data

In [8]:
preprocessor = ColumnTransformer(
    transformers = [
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

### Step3. Model Learning and the model evaluation

In [9]:
# Define model
model = RandomForestRegressor(n_estimators=100, random_state=0)

# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

# Preprocessing of training data, fit model 
clf.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = clf.predict(X_valid)

print('MAE:', mean_absolute_error(y_valid, preds))

MAE: 17740.290308219177


### Step 4. Improve the performance

In [11]:
# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='most_frequent')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps = [('imputer', SimpleImputer(strategy='most_frequent')), ('onehot', OneHotEncoder(handle_unknown='ignore'))] )

# Bundle preprocessings for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers = [
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# Define model
model = RandomForestRegressor(n_estimators=100, random_state=1)

In [12]:
pipeline_bundle = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

pipeline_bundle.fit(X_train, y_train)
preds = pipeline_bundle.predict(X_valid)

error_score = mean_absolute_error(y_valid, preds)
print('MAE:',error_score)

MAE: 17544.61982876712


### Step 5. Generate test predictors

In [13]:
preds_test = pipeline_bundle.predict(X_test)

### Step 6. Save results to CSV file

In [14]:
output = pd.DataFrame({'Id': X_test.index, 'SalePrice': preds_test})
output.to_csv('prediction_results.csv', index=False)